In [70]:
import os
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import HDBSCAN
from sklearn.metrics.pairwise import haversine_distances

# Important for conversion of lat-long based distance to meters
RADIUS_OF_EARTH_AT_SPACE_NEEDLE = 6366.512563943 # km

# Set seed
np.random.seed(42)

In [71]:
# Random helpers + custom distance metrics 
def extract_yearly_data(df, dirname, base_filename, year_range):
    dirpath = os.path.join(os.getcwd(), dirname)
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
    path = os.path.join(dirpath, base_filename)
    for year in year_range:
        year_df = df.loc[df["Year"] == year]
        year_df.to_csv(path + "_year_" + str(year) + ".csv", index=False)
    return

def top_block_counts(crime_df, n):
    """List in ascending order"""
    top_blocks = (
        crime_df.groupby(["Latitude", "Longitude"])
        .size()
        .to_frame(name="count")
        .reset_index()
        .sort_values(by=["count"], ascending=True)
        .tail(n)
    )
    return top_blocks["count"].values

def meters_to_hav(meters, R=RADIUS_OF_EARTH_AT_SPACE_NEEDLE):
    """Converts a distance in meters to haversine distance"""
    hav = meters / (R * 1000)
    return hav

def haversine_np(lon1, lat1, lon2, lat2, R=RADIUS_OF_EARTH_AT_SPACE_NEEDLE):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = R * c
    return km

def linearized_haversine(a, b, R=RADIUS_OF_EARTH_AT_SPACE_NEEDLE):
    a_rad = a * (2 * np.pi / 360)
    b_rad = b * (2 * np.pi / 360)
    x_1, y_1 = a_rad
    x_2, y_2 = b_rad
    delta_x = R * np.cos(y_1) * (x_2 - x_1)
    delta_y = R * (y_2 - y_1)
    return np.sqrt(delta_x**2 + delta_y**2)

In [72]:
def get_windows_months(start_year, end_year, length_months=18, step_months=6):
    num_months = (end_year - start_year + 1) * 12
    starts = range(1, num_months + 1, step_months)
    ends = [min(start + length_months - 1, num_months) for start in starts]
    windows_in_months = list(zip(starts, ends))
    if length_months == step_months:
        return windows_in_months
    else:
        return windows_in_months[: -(int(length_months / step_months) - 1)]

def to_month_year(months, base_year):
    year = base_year + int((months - 1) / 12)
    month = ((months - 1) % 12) + 1
    return month, year

def get_windows(start_year, end_year, length_months=18, step_months=6):
    windows_in_months = get_windows_months(
        start_year, end_year, length_months=length_months, step_months=step_months
    )
    windows = [
        (to_month_year(start, start_year), to_month_year(end, start_year))
        for start, end in windows_in_months
    ]
    return windows

def extract_windows(df, windows):
    extracts = []
    for window in windows:
        start, end = window
        start_month, start_year = start
        end_month, end_year = end
        # Get crimes in year range
        df_years = df.loc[(df["Year"] >= start_year) & (df["Year"] <= end_year)]
        # Remove crimes in start year out of range
        df_filter1 = df_years.loc[
            ~((df_years["Year"] == start_year) & (df_years["Month"] < start_month))
        ]
        # Remove crimes in end year out of range
        df_window = df_filter1.loc[
            ~((df_filter1["Year"] == end_year) & (df_filter1["Month"] > end_month))
        ]
        extracts.append((window, df_window))
    return extracts

In [166]:
def grid_search_hdbscans(windows, alphas, max_size_mult, max_eps_m=150):
    """
    Search for a decent, stable alpha for each n_hood -> df + summaries
    Labels stored in output df. Need to write it to disk after for Tableau.
    """
    run_summaries = defaultdict(list)
    clustered_dfs = defaultdict(list)
    max_eps_hav = meters_to_hav(max_eps_m)
    # For each alpha, cluster each window
    for alpha in alphas:
        alpha = round(alpha, 5)
        colname = "cluster_labs"
        print(f"Clustering with alpha={alpha}")
        for i, (_, df) in enumerate(windows):
            output_df = df.copy()
            X = output_df[["lat_rad", "long_rad"]]
            crime_in_window = X.shape[0]
            min_samples = int(alpha * crime_in_window)
            # HDBSCAN
            hdb = HDBSCAN(
                cluster_selection_epsilon=max_eps_hav,
                min_samples=min_samples,
                min_cluster_size=2 * min_samples,
                max_cluster_size=int(max_size_mult) * min_samples,
                metric="haversine",
                store_centers="centroid",
            ).fit(X)
            labels_hdb_ = hdb.labels_
            output_df[colname] = labels_hdb_
            # Count clusters and noise
            num_clusters_ = len(set(labels_hdb_)) - (1 if -1 in labels_hdb_ else 0)
            num_noise_ = list(labels_hdb_).count(-1)
            percent_clustered_ = (crime_in_window - num_noise_) / crime_in_window
            # Store df and summary
            run_summaries[alpha].append((i, num_clusters_, percent_clustered_))
            clustered_dfs[alpha].append(output_df)
    return run_summaries, clustered_dfs


def grid_search_epsilons(windows, alpha, max_epsilons):
    """
    given a fixed alpha, refine max_eps -> df + summaries
    Labels stored in output df. Need to write it to disk for Tableau.
    """
    run_summaries = defaultdict(list)
    clustered_dfs = defaultdict(list)
    alpha = round(alpha, 5)
    # For each max_eps, cluster each window
    for max_eps_m in max_epsilons:
        colname = "clust_eps=" + str(max_eps_m)
        print(f"Clustering with max_eps={max_eps_m}")
        max_eps_hav = meters_to_hav(max_eps_m)
        for i, (_, df) in enumerate(windows):
            output_df = df.copy()
            X = output_df[["lat_rad", "long_rad"]]
            crime_in_window = X.shape[0]
            min_samples = int(alpha * crime_in_window)
            # HDBSCAN
            hdb = HDBSCAN(
                cluster_selection_epsilon=max_eps_hav,
                min_samples=min_samples,
                min_cluster_size=min_samples,
                metric="haversine",
                store_centers="centroid",
            ).fit(X)
            labels_hdb_ = hdb.labels_
            output_df[colname] = labels_hdb_
            # Count clusters and noise
            num_clusters_ = len(set(labels_hdb_)) - (1 if -1 in labels_hdb_ else 0)
            num_noise_ = list(labels_hdb_).count(-1)
            percent_clustered_ = (crime_in_window - num_noise_) / crime_in_window
            # Store df and summary
            run_summaries[max_eps_m].append((i, num_clusters_, percent_clustered_))
            clustered_dfs[max_eps_m].append(output_df)
    return run_summaries, clustered_dfs


def plot_summaries_alphas(run_summaries):
    num_plots = len(run_summaries)
    fig, axes = plt.subplots(num_plots, 2, figsize=(36, 12))
    axes = axes.flatten()  # Flatten the axes for easier iteration
    for i, alpha in enumerate(run_summaries):
        arr = np.array(run_summaries[alpha])
        windows = arr[:, 0].astype(int)
        cluster_counts = arr[:, 1]
        percentages_clustered = arr[:, 2]
        ax1, ax2 = axes[i * 2], axes[i * 2 + 1]  # Get subplots for current plot
        # Plot nclusters vs. windows
        ax1.plot(windows, cluster_counts)
        ax1.set_xticks(windows)
        ax1.set_xlabel("Window")
        ax1.set_ylabel("Number of Clusters")
        ax1.set_title(f"Cluster counts by window: alpha={alpha}")
        # Plot percent_clustered vs. windows
        ax2.plot(windows, percentages_clustered)
        ax2.set_xticks(windows)
        ax2.set_xlabel("Window")
        ax2.set_ylabel("% Crime Clustered")
        ax2.set_title(f"% Clustered by window: alpha={alpha}")
    plt.tight_layout()
    plt.show()
    return


def plot_summaries_eps(run_summaries):
    num_plots = len(run_summaries)
    fig, axes = plt.subplots(num_plots, 2, figsize=(24, 12))
    axes = axes.flatten()  # Flatten the axes for easier iteration
    for i, eps in enumerate(run_summaries):
        arr = np.array(run_summaries[eps])
        windows = arr[:, 0].astype(int)
        cluster_counts = arr[:, 1]
        percentages_clustered = arr[:, 2]
        ax1, ax2 = axes[i * 2], axes[i * 2 + 1]  # Get subplots for current plot
        # Plot nclusters vs. windows
        ax1.plot(windows, cluster_counts)
        ax1.set_xticks(windows)
        ax1.set_xlabel("Window")
        ax1.set_ylabel("Number of Clusters")
        ax1.set_title(f"Cluster counts by window: eps={eps}")
        # Plot percent_clustered vs. windows
        ax2.plot(windows, percentages_clustered)
        ax2.set_xticks(windows)
        ax2.set_xlabel("Window")
        ax2.set_ylabel("% Crime Clustered")
        ax2.set_title(f"% Clustered by window: eps={eps}")
    plt.tight_layout()
    plt.show()
    return

In [74]:
def grid_search_all_neighborhoods_alpha(
    crime_df, sectors_to_cluster, window_times, alphas, max_eps_m=100
):
    all_summaries = {}
    all_dfs = {}
    for sector in sectors_to_cluster:
        print(f"Clustering sector {sector}")
        df = crime_df.loc[crime_df["Sector"] == sector]
        windows = extract_windows(df, window_times)
        run_summaries, clustered_dfs = grid_search_hdbscans(
            windows, alphas, max_eps_m=max_eps_m
        )
        all_summaries[sector] = run_summaries
        all_dfs[sector] = clustered_dfs
    return all_summaries, all_dfs


def grid_search_all_neighborhoods_eps(
    crime_df, sectors_to_cluster, window_times, best_alphas, epsilons
):
    all_summaries = {}
    all_dfs = {}
    for sector in sectors_to_cluster:
        alpha = best_alphas[sector]
        print(f"Clustering sector {sector}")
        df = crime_df.loc[crime_df["Sector"] == sector]
        windows = extract_windows(df, window_times)
        run_summaries, clustered_dfs = grid_search_epsilons(
            windows, alpha, epsilons
        )
        all_summaries[sector] = run_summaries
        all_dfs[sector] = clustered_dfs
    return all_summaries, all_dfs

In [77]:
# Read the cleaned data
crime_df = pd.read_csv("../../cleaned_SPD_Crime_Data.csv")
crime_df = crime_df.loc[crime_df["Year"] < 2024]

In [78]:
# Extract the relevant clusters
sectors = crime_df["Sector"].value_counts().index
sectors_to_cluster = sectors.values[:17]
sectors_to_cluster

array(['U', 'B', 'E', 'M', 'K', 'D', 'Q', 'R', 'L', 'N', 'J', 'W', 'S',
       'F', 'C', 'G', 'O'], dtype=object)

In [79]:
# Cluster year by year for now
window_times = get_windows(2008, 2023, length_months=12, step_months=12)
# alphas = np.linspace(0.005, 0.03, 11)  # alphas to test
# all_summaries, all_dfs = grid_search_all_neighborhoods_alpha(
#     crime_df, sectors_to_cluster, window_times, alphas
# )

In [ ]:
# Can check the run summaries to choose alphas by neighborhood
# run_summaries = all_summaries['O']
# plot_summaries_alphas(run_summaries)

In [ ]:
# Can write individual windows to disk here
# clustered_dfs_alphas = all_dfs['K']
# clustered_dfs_alphas[.02][0].to_csv("test.csv")

In [14]:
"""
These are the alphas that resulted in the most stable small clusters across all windows.
Unfortunately, even with these alphas there are still windows with degenrate clusters.
I plan to manually fix these bad clusterings by setting a different alpha/eps for each
problem window, as needed. The degree to which I'll focus on that will depend
on how important those windows end up being to our analysis.
"""

my_favorite_alphas = {
    "U": 0.015,
    "B": 0.01,
    "E": 0.025,
    "M": 0.03,
    "K": 0.025,
    "D": 0.015,
    "Q": 0.015,
    "R": 0.025,
    "L": 0.02,
    "N": 0.0225,
    "J": 0.02,
    "W": 0.02,
    "S": 0.03,
    "F": 0.02,
    "C": 0.015,
    "G": 0.02,
    "O": 0.025,
}

In [330]:
# Try restricting epsilon to reduce number of big clusters
# max_epsilons = [50, 75, 100]
# all_summaries_eps, all_dfs_eps = grid_search_all_neighborhoods_eps(
#     crime_df, sectors_to_cluster, window_times, my_favorite_alphas, max_epsilons
# )

Clustering sector U
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector B
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector E
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector M
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector K
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector D
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector Q
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector R
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector L
Clustering with max_eps=50
Clustering with max_eps=75
Clustering with max_eps=100
Clustering sector N
Clustering with max_eps=50
Clustering with max_eps=75
Clusteri

In [ ]:
# Can check here for epsilon selection after alpha is chosen
# run_summaries_eps = all_summaries_eps['O']
# plot_summaries_eps(run_summaries_eps)

In [356]:
# Can write output by sector here
# clustered_dfs_eps = all_dfs_eps['E']
# clustered_dfs_eps[50][0].to_csv("test.csv")

In [327]:
"""
These are the 'best' alpha/epsilon combinations for each window. Dialing
back the max_eps oftentimes did not affect cluster stability, but
sometimes it did actually improve it.
"""

best_params = {
    "U": (0.015, 100, 7),
    "B": (0.01, 100, 10),
    "E": (0.025, 75, 10),
    "M": (0.02, 65, 50),
    "K": (0.025, 75, 10),
    "D": (0.0125, 75, 17),
    "Q": (0.01, 11, 30),
    "R": (0.015, 20, 15),
    "L": (0.02, 50, 10),
    "N": (0.0225, 50, 10),
    "J": (0.02, 50, 10),
    "W": (0.015, 40, 30),
    "S": (0.02, 50, 30),
    "F": (0.02, 50, 10),
    "C": (0.015, 50, 10),
    "G": (0.02, 75, 10),
    "O": (0.025, 50, 10),
}

In [86]:
def cluster_all_neighborhoods(crime_df, sectors_to_cluster, window_times, best_params):
    all_summaries = {}
    all_dfs = {}
    for sector in sectors_to_cluster:
        alpha, max_eps_m, max_size_mult = best_params[sector]
        print(f"Clustering sector {sector} with alpha={alpha} and max_eps={max_eps_m}")
        df = crime_df.loc[crime_df["Sector"] == sector]
        windows = extract_windows(df, window_times)
        run_summaries, clustered_dfs = grid_search_hdbscans(
            windows, [alpha], max_size_mult, max_eps_m=max_eps_m
        )
        all_summaries[sector] = run_summaries
        all_dfs[sector] = clustered_dfs
    return all_summaries, all_dfs

In [328]:
all_summaries, all_dfs = cluster_all_neighborhoods(
    crime_df, sectors_to_cluster, window_times, best_params
)

Clustering sector U with alpha=0.015 and max_eps=100
Clustering with alpha=0.015
Clustering sector B with alpha=0.01 and max_eps=100
Clustering with alpha=0.01
Clustering sector E with alpha=0.025 and max_eps=75
Clustering with alpha=0.025
Clustering sector M with alpha=0.02 and max_eps=65
Clustering with alpha=0.02
Clustering sector K with alpha=0.025 and max_eps=75
Clustering with alpha=0.025
Clustering sector D with alpha=0.0125 and max_eps=75
Clustering with alpha=0.0125
Clustering sector Q with alpha=0.01 and max_eps=11
Clustering with alpha=0.01
Clustering sector R with alpha=0.015 and max_eps=20
Clustering with alpha=0.015
Clustering sector L with alpha=0.02 and max_eps=50
Clustering with alpha=0.02
Clustering sector N with alpha=0.0225 and max_eps=50
Clustering with alpha=0.0225
Clustering sector J with alpha=0.02 and max_eps=50
Clustering with alpha=0.02
Clustering sector W with alpha=0.015 and max_eps=40
Clustering with alpha=0.015
Clustering sector S with alpha=0.02 and max_

In [95]:
def concatenate_results(all_dfs, best_params):
    by_neighborhood = []
    for sector in all_dfs:
        alpha, _, _= best_params[sector]
        clustered_windows = all_dfs[sector][alpha]
        joined_df = pd.concat(clustered_windows, axis=0)
        by_neighborhood.append(joined_df)
    all_neighborhoods = pd.concat(by_neighborhood, axis=0)
    return all_neighborhoods

In [329]:
all_neighborhoods = concatenate_results(all_dfs, best_params)
all_neighborhoods.to_csv("test.csv")

## Began reclustering here
Some windows, even after tuning, had degenerate clusters. We try to improve those clusters here. Post-processing will take care of some of the problems, but not all of them, so we it's worth it to create some better raw materials.

In [12]:
# Read output from previous cluster
# all_neighborhoods = pd.read_csv("test.csv")

In [25]:
# These are the sectors that had windows with degenerate clusters 
# sectors_to_fix = ["B", "C", "D", "M", "N", "O", "Q", "W"]

In [62]:
# Our approach is manual. We select new alphas/eps, then rerun just on that sector
def recluster_sector(sector, crime_df, window_times, best_params):
    _, recluster_dfs = cluster_all_neighborhoods(
        crime_df, [sector], window_times, best_params
    )
    recluster_df = concatenate_results(recluster_dfs, best_params)
    recluster_df.to_csv("test.csv")
    return

Clustering with alpha=0.02
Clustering with alpha=0.0225
Clustering with alpha=0.025
Clustering with alpha=0.0275
Clustering with alpha=0.03
Clustering with alpha=0.0325
Clustering with alpha=0.035
Clustering with alpha=0.0375
Clustering with alpha=0.04
Clustering with alpha=0.02
Clustering with alpha=0.0225
Clustering with alpha=0.025
Clustering with alpha=0.0275
Clustering with alpha=0.03
Clustering with alpha=0.0325
Clustering with alpha=0.035
Clustering with alpha=0.0375
Clustering with alpha=0.04
Clustering with alpha=0.02
Clustering with alpha=0.0225
Clustering with alpha=0.025
Clustering with alpha=0.0275
Clustering with alpha=0.03
Clustering with alpha=0.0325
Clustering with alpha=0.035
Clustering with alpha=0.0375
Clustering with alpha=0.04
Clustering with alpha=0.02
Clustering with alpha=0.0225
Clustering with alpha=0.025
Clustering with alpha=0.0275
Clustering with alpha=0.03
Clustering with alpha=0.0325
Clustering with alpha=0.035
Clustering with alpha=0.0375
Clustering with

In [ ]:
# recluster_sector("W", crime_df, window_times, best_params)